In [1]:
# 初始化环境
import os

os.environ['OPENAI_API_KEY'] = '${YOUR_API_KEY}'



In [2]:
# 一般参数
from openai import OpenAI
from openai.types.chat import completion_create_params

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! "}
    ],
    n=1,
    max_tokens=2048,
    temperature=0.8,
    # top_p=1,
    stop="!",
    # seed=1234567,
    user="user_1234567",
)

print(completion.model_dump_json())

{"id":"chatcmpl-9ErR52JmnfipEu4QVdSN1ThNO39ng","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"Hello","role":"assistant","function_call":null,"tool_calls":null}}],"created":1713329027,"model":"gpt-3.5-turbo-1106","object":"chat.completion","system_fingerprint":"fp_77a673219d","usage":{"completion_tokens":1,"prompt_tokens":20,"total_tokens":21}}


In [11]:
# response_format 参数 https://platform.openai.com/docs/guides/text-generation/json-mode
from openai import OpenAI
from openai.types.chat import completion_create_params

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": "Who won the world series in 2020?"}
    ],
    # 对content进行json格式化
    response_format=completion_create_params.ResponseFormat(type="json_object")
)

print(completion.choices[0].message.model_dump_json())

{"content":"{\n  \"winner\": \"Los Angeles Dodgers\"\n}","role":"assistant","function_call":null,"tool_calls":null}


response_format 参数 https://platform.openai.com/docs/guides/text-generation/json-mode
如果想使用response_type为json格式，必须在prompt中声明‘json’字段，否则报错：
`'messages' must contain the word 'json' in some form, to use 'response_format' of type 'json_object'`
这是个反例

In [12]:
from openai import OpenAI
from openai.types.chat import completion_create_params

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Who won the world series in 2020?"}
    ],
    # 对content进行json格式化
    response_format=completion_create_params.ResponseFormat(type="json_object")
)

print(completion.choices[0].message.model_dump_json())

BadRequestError: Error code: 400 - {'error': {'message': "'messages' must contain the word 'json' in some form, to use 'response_format' of type 'json_object'. (request id: 20240416163749766878375Ak7gj9S1)", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

stream 是一个一个蹦的

In [14]:
# stream流式输出
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ],
    stream=True
)

for chunk in completion:
    print(chunk.choices[0].delta.model_dump_json())

{"content":"","function_call":null,"role":"assistant","tool_calls":null}
{"content":"Hello","function_call":null,"role":null,"tool_calls":null}
{"content":"!","function_call":null,"role":null,"tool_calls":null}
{"content":" How","function_call":null,"role":null,"tool_calls":null}
{"content":" can","function_call":null,"role":null,"tool_calls":null}
{"content":" I","function_call":null,"role":null,"tool_calls":null}
{"content":" assist","function_call":null,"role":null,"tool_calls":null}
{"content":" you","function_call":null,"role":null,"tool_calls":null}
{"content":" today","function_call":null,"role":null,"tool_calls":null}
{"content":"?","function_call":null,"role":null,"tool_calls":null}
{"content":null,"function_call":null,"role":null,"tool_calls":null}


In [27]:
# logit_bias参数 [-100, 100], 100表示必选，-100 表示禁止
from openai import OpenAI
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("Hello"))

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ],
    logit_bias={enc.encode("Hello")[0]: -100} # 这里要注意一个点：key是token
)

print(completion.choices[0].message.model_dump_json())

[9906]
{"content":"Hi there! How can I assist you today?","role":"assistant","function_call":null,"tool_calls":null}


In [5]:
# logprobs参数
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "user", "content": "Hello!"}
    ],
    logprobs=True,
    top_logprobs=2
)

print(completion.choices[0].message.model_dump_json())
print(completion.choices[0].logprobs.model_dump_json())

{"content":"Hello! How can I assist you today?","role":"assistant","function_call":null,"tool_calls":null}
{"content":[{"token":"Hello","bytes":[72,101,108,108,111],"logprob":-0.21570958,"top_logprobs":[{"token":"Hello","bytes":[72,101,108,108,111],"logprob":-0.21570958},{"token":"Hi","bytes":[72,105],"logprob":-1.6720008}]},{"token":"!","bytes":[33],"logprob":-0.20926832,"top_logprobs":[{"token":"!","bytes":[33],"logprob":-0.20926832},{"token":" there","bytes":[32,116,104,101,114,101],"logprob":-1.6910303}]},{"token":" How","bytes":[32,72,111,119],"logprob":-0.00095136015,"top_logprobs":[{"token":" How","bytes":[32,72,111,119],"logprob":-0.00095136015},{"token":" Is","bytes":[32,73,115],"logprob":-7.526584}]},{"token":" can","bytes":[32,99,97,110],"logprob":-0.014271167,"top_logprobs":[{"token":" can","bytes":[32,99,97,110],"logprob":-0.014271167},{"token":" are","bytes":[32,97,114,101],"logprob":-4.673554}]},{"token":" I","bytes":[32,73],"logprob":-0.00001700133,"top_logprobs":[{"tok

tools

In [3]:
# 1. 定义函数
# 1.1 定义模拟获取天气的本地函数
def get_current_weather(location, unit):
    # Call the weather API
    return f"It's 20 {unit} in {location}"


# 1.2 定义函数字典方便调用
function_dict = {
    "get_current_weather": get_current_weather,
}

# 1.3 定义chat接口需要的函数
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    }
]

messages = [
    {"role": "user", "content": "What's the weather like in Boston today with celsius and fahrenheit?"}]

client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)

print(f"first call: {completion.model_dump_json()}") # 2. 第一次调用chat接口，返回的是函数调用的提示


first call: {"id":"chatcmpl-9ErRRLw7IbrkWFODggnUzwk99QkFR","choices":[{"finish_reason":"tool_calls","index":0,"logprobs":null,"message":{"content":null,"role":"assistant","function_call":null,"tool_calls":[{"id":"call_ToIgUbw4UcZhi6dyAyCPyf0v","function":{"arguments":"{\"location\": \"Boston, MA\", \"unit\": \"celsius\"}","name":"get_current_weather"},"type":"function"},{"id":"call_XrMEOIxoy6JUSoSJ1DuQuQrc","function":{"arguments":"{\"location\": \"Boston, MA\", \"unit\": \"fahrenheit\"}","name":"get_current_weather"},"type":"function"}]}}],"created":1713329049,"model":"gpt-3.5-turbo-1106","object":"chat.completion","system_fingerprint":"fp_ad2b9c6e11","usage":{"completion_tokens":59,"prompt_tokens":89,"total_tokens":148}}


response中从传入的tools中“智能”得选出了应该调用的函数
```
"tool_calls": [{
				"id": "call_gUDAqeorgDQjWJtvt0sxevcl",
				"function": {
					"arguments": "{\"location\": \"Boston, MA\", \"unit\": \"celsius\"}",
					"name": "get_current_weather"
				},
				"type": "function"
			}, {
				"id": "call_lZEhwk2r2HliYy54ZFBhdCpU",
				"function": {
					"arguments": "{\"location\": \"Boston, MA\", \"unit\": \"fahrenheit\"}",
					"name": "get_current_weather"
				},
				"type": "function"
			}]
```

In [4]:
import json

# 3. 从结果接口的结果中获取函数调用的参数 进行本地函数调用
# 3.1 获取函数调用的参数
response_message = completion.choices[0].message
messages.append(response_message)   # 必须要传
# 遍历tool_calls，获取函数调用的id，函数名，函数参数
for tool_call in response_message.tool_calls:
    tool_id = tool_call.id
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    # 3.2 调用本地函数
    function_response = function_dict.get(function_name)(**function_args)
    # 3.3 将本地函数的结果作为chat接口的输入
    messages.append({
        "role": "tool",
        "tool_call_id": tool_id,
        "content": function_response,
    })

def custom_encoder(obj):
    if hasattr(obj, 'model_dump'):
        return obj.model_dump()  # 调用自定义的 to_dict 方法
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

print(f"second call massages: {json.dumps(messages, default=custom_encoder,ensure_ascii=False)}")

second call massages: [{"role": "user", "content": "What's the weather like in Boston today with celsius and fahrenheit?"}, {"content": null, "role": "assistant", "function_call": null, "tool_calls": [{"id": "call_ToIgUbw4UcZhi6dyAyCPyf0v", "function": {"arguments": "{\"location\": \"Boston, MA\", \"unit\": \"celsius\"}", "name": "get_current_weather"}, "type": "function"}, {"id": "call_XrMEOIxoy6JUSoSJ1DuQuQrc", "function": {"arguments": "{\"location\": \"Boston, MA\", \"unit\": \"fahrenheit\"}", "name": "get_current_weather"}, "type": "function"}]}, {"role": "tool", "tool_call_id": "call_ToIgUbw4UcZhi6dyAyCPyf0v", "content": "It's 20 celsius in Boston, MA"}, {"role": "tool", "tool_call_id": "call_XrMEOIxoy6JUSoSJ1DuQuQrc", "content": "It's 20 fahrenheit in Boston, MA"}]


In [5]:
# 4. 第二次调用chat接口，返回的是chat的最终结果
completion_final = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
)
print(f"second call: {completion_final.model_dump_json()}")

second call: {"id":"chatcmpl-9ErSyoQdo5sFRvXDy2nATr13qrEtA","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"The weather in Boston today is 20 degrees Celsius (68 degrees Fahrenheit).","role":"assistant","function_call":null,"tool_calls":null}}],"created":1713329144,"model":"gpt-3.5-turbo-1106","object":"chat.completion","system_fingerprint":"fp_ad2b9c6e11","usage":{"completion_tokens":15,"prompt_tokens":115,"total_tokens":130}}


小结tool
tools的使用会调用OpenAI两次，第一次将本地函数、需求传入，返回要调用的函数(可能是多个，后面可以遍历调用)；本地逐一调用函数并将结果与第一步返回的函数列表(包含`tool_calls`的外层)一起传入openAI，让其组装最后的结果。这里必须要传tool_calls的外层给openAI，不然它不知道如何组装本地函数调用结果，会提示你要带着tool_calls: `Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'`

vision
将视觉能力融合到自己的场景中，例如图像识别、智能监控等。

In [ ]:
# 4. 第二次调用chat接口，返回的是chat的最终结果
completion_final = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
)
print(f"second call: {completion_final.model_dump_json()}")

second call: {"id":"chatcmpl-9ErSyoQdo5sFRvXDy2nATr13qrEtA","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"The weather in Boston today is 20 degrees Celsius (68 degrees Fahrenheit).","role":"assistant","function_call":null,"tool_calls":null}}],"created":1713329144,"model":"gpt-3.5-turbo-1106","object":"chat.completion","system_fingerprint":"fp_ad2b9c6e11","usage":{"completion_tokens":15,"prompt_tokens":115,"total_tokens":130}}


In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0].model_dump_json())


{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"The image shows a beautiful natural landscape with a wooden boardwalk extending straight into the distance. The boardwalk is surrounded by lush green grasses and foliage. The sky is bright and partly cloudy, suggesting a sunny day with good weather. In the distance, you can see a line of trees and some shrubbery that demarcates the end of the open field. This looks like a peaceful setting for a walk or to enjoy nature.","role":"assistant","function_call":null,"tool_calls":null}}


[后记]api中注意到的参数：

* frequency_penalty，频率惩罚，[-2,2]，负值时会奖励重复，正值会惩罚重复。

* presence_penalty，[-2,2]，是否重复使用之前生成的词汇，正值会惩罚新token，即正值会唠叨，负值会更灵活。若想就围绕一个话题使劲地讨论 -2，若想避免重复讨论 2。

* logit_bias 增加或减少某个token出现的概率，-100 禁止 100必选。

* response_format 我上次在这里踩了坑：单纯指定这个参数不行的，需要在prompt中声明json，API文档中也强调了这点(不声明的话报错：'messages' must contain the word 'json' in some form, to use 'response_format' of type 'json_object')。

  > 忘记之前的运行效果了，这次试的效果是：response_type可以不指定 prompt中指定即可；但response_type指定了，prompt必须指定json。

* seed，和SD中的seed类同。